# TCDM Práctica 3: MapReduce con *datasets* reales (StackOverflow), curso 25-26.

**Objetivo**: Implementar algoritmos de MapReduce usando **MrJob** para analizar datos reales de StackOverflow.

### Descarga de los datos

In [1]:
import os
import tarfile
import urllib.request

# URLs de descarga
file: str = "stackoverflow.csv.tar.gz"
URL: str = f"https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/{file}"

# Descargar el fichero tar.gz
urllib.request.urlretrieve(URL, file)
assert os.path.isfile(file), f"Error: {file} no se ha descargado correctamente."

# Extraer archivos
with tarfile.open("stackoverflow.csv.tar.gz", "r:gz") as tar:
    for member in tar.getmembers():
        if member.name.endswith(("Posts.csv", "Users.csv")):
            tar.extract(member)

/tmp/ipykernel_619974/1726189557.py:17: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extract(member)


## Configuración inicial


La configuración inicial incluye la creación de un entorno virtual (si no lo tenéis creado ya) y la descargar de los paquetes necesarios.

> **OJO**: Todo esto hay que hacerlo **dentro del contenedor `namenode`, como usuario `luser`**, que es el encargado de lanzar los trabajos MapReduce como un usuario normal.

```bash
test -e ~/.venv || python3 -m venv ~/.venv
. ~/.venv/bin/activate
# Descargar requirements
wget -q https://github.com/dsevilla/tcdm-public/raw/25-26/practicas/p3/requirements.txt
cat requirements.txt
# Instalar requirements
pip3 install -r requirements.txt
```

## Ejercicio 1: Contar cuántas respuestas tiene cada pregunta, después devolverlas ordenadas por número de respuestas

**Entrada**: `Posts.csv`

**Salida**: `(pregunta_id, número_respuestas)`

**Lógica**: 
- Las respuestas tienen `PostTypeId=2` 
- El `ParentId` de una respuesta apunta a la pregunta
- Contar agrupando por `ParentId`

**Ejercicio**: Terminar la implementación en `p3_so_count_answers.py`:

```bash
python3 so_count_answers.py Posts.csv > so_count_answers.out
# (o bien si se ejecuta dentro del clúster Hadoop)
python3 so_count_answers.py -r hadoop hdfs://namenode:9000/user/luser/.../Posts.csv > so_count_answers.out
```

### El problema de la ordenación:

El resultado anterior no está ordenado. ¿Por qué? Dentro de cada reducer, las claves están ordenadas lexicográficamente, pero la salida global concatena resultados de múltiples reducers sin garantía de orden. Para obtener las preguntas con más respuestas en un resultado ordenado, tenemos varias opciones:

- Ordenar como último paso con un programa externo. Esto tiene el problema de que necesitamos otra herramienta externa, tipo `sort` de UNIX/Linux o hacerlo en `pandas`, etc. Se puede realizar, pero tiene el problema de que no es escalable.

- Hacer dos pasos de MapReduce. En el primer paso contamos las respuestas, y en el segundo ordenamos los resultados. Para ello debemos especificar que sólo hay un *reducer*, que recibe todas las claves y puede ordenarlas y escribirlas ordenadas como resultado. Problema: ¿Qué pasa si el resultado es muy grande? Un sólo *reducer* puede no ser capaz de tratar todos los datos. En general esto no es un problema, porque los trabajos de MapReduce suelen reducir mucho el tamaño de los datos. Pero es un problema a tener en cuenta. En nuestro caso lo será, porque si hay millones de preguntas, el resultado puede ser muy grande.

- Hacer un Top-K integrado. En este caso, cada *mapper* y *reducer* mantiene una lista de los K elementos más grandes que ha visto. Se elige un conjunto de *buckets* tal que la división en esos *buckets* no llega a ser más grande que la memoria. Cada uno de los *reducers* calcula su top-k. Al final, un único *reducer* recibe las listas de los demás y calcula el Top-K global. Esto es escalable, porque el tamaño del resultado es siempre K, que es pequeño. El problema es que no obtenemos el resultado completo, sólo los K primeros.

En general la ordenación no es un problema, porque lo que queremos es que se produzcan **todos los resultados correctos**, no el orden.

Veamos cómo cambiar el programa para implementar estas opciones.

#### Dos pasos MapReduce para ordenar

Cambia el programa `p3_so_count_answers.py` para hacer dos pasos de MapReduce y ordenar el resultado:

```bash
python3 so_count_answers_ordered.py Posts.csv > respuestas_ordenadas.out
```

#### Top-K

Cambia el programa `p3_so_count_answers.py` para implementar el Top-K. El programa debe aceptar dos parámetros: `--top-k` (número de elementos a devolver) y `--num-buckets` (número de *buckets* para dividir los datos). Por ejemplo, para obtener las 20 preguntas con más respuestas usando 100 *buckets*:

```bash
python3 so_count_answers_topk.py --top-k=20 --num-buckets=100 Posts.csv > top20_respuestas.out
```

## Ejercicio 2: Filtrado y agregación

**Objetivo**: Contar preguntas agrupadas por año y etiqueta.

**Entrada**: `Posts.csv`

**Salida**: `((año, etiqueta), número_preguntas)`

**Lógica**:
- Filtrar solo preguntas (`PostTypeId=1`)
- Extraer año de `CreationDate` 
- Parsear etiquetas del campo `Tags` (formato: `<python><pandas><web>`)
- Contar por cada par (año, etiqueta)

**Ejecución**:

```bash
python3 so_bytagyear.py Posts.csv > preguntas_por_tag_año.out
```

**Salida esperada**:
```
(2020,python)     245
(2019,javascript) 189
(2021,java)       156
...
```


## Ejercicio 3: JOIN simple

**Objetivo**: Relacionar preguntas con la reputación de sus autores.

**Entrada**: `Posts.csv` y `Users.csv`

**Salida**: `(pregunta_id, (reputación_autor, nombre_autor))`

**Lógica**:
- Map-side join: usar prefijos para distinguir tipos de entrada
- Crear archivos prefijados: `P|<línea_post>` y `U|<línea_user>`
- En mapper: agrupar por `OwnerUserId`
- En reducer: combinar datos de usuario y preguntas

**Preparación**:
```bash
sed 's/^/P|/' Posts.csv > Posts_prefixed.csv
sed 's/^/U|/' Users.csv > Users_prefixed.csv
```

> **OJO**: Si se ejecuta en Hadoop, hay que copiar los archivos a HDFS:
> ```bash
> hdfs dfs -put Posts_prefixed.csv /user/luser/...
> hdfs dfs -put Users_prefixed.csv /user/luser/...
> ```

**Ejecución**:

```bash
python3 so_join.py Posts_prefixed.csv Users_prefixed.csv > pregunta_reputacion.out
```

**Salida esperada**:
```
495829        (39,Daniel)
496121        (39,Daniel)
496183        (39,Daniel)
...
```

## Ejercicio 4: Coocurrencia de etiquetas

**Objetivo**: Encontrar pares de etiquetas que aparecen juntas frecuentemente.

**Entrada**: `Posts.csv`

**Salida**: `((etiqueta1, etiqueta2), frecuencia)`

**Lógica**:
- Para cada pregunta extraer sus múltiples etiquetas
- Generar todas las combinaciones de pares 
- Contar frecuencia de cada par

**Ejecución**:

```bash
python3 so_tagcooc.py Posts.csv > coocurrencia_tags.out
```

**Salida esperada**:
```
python,pandas     89
javascript,html   76
java,spring       54
...
```

## Ejercicio 5: Análisis temporal avanzado

**Objetivo**: Evolución de popularidad de tecnologías en el tiempo.

**Paso 1**: Actividad por tecnología y mes
- **Entrada**: `Posts.csv`
- **Salida temporal**: `((etiqueta, año-mes), actividad)` donde actividad = preguntas + respuestas × 0.5

**Paso 2**: Calcular tendencias (crecimiento mes a mes)
- **Entrada**: actividad del paso 1
- **Salida temporal**: `((etiqueta, mes), delta_actividad)`

**Paso 3**: Clasificar tecnologías por tendencia
- **Entrada**: tendencias del paso 2
- **Criterio**: promedio de delta últimos 6 meses
- **Salida final**: tecnologías en crecimiento/declive

**Ejecución**:

```bash
python3 so_tecnology_evolution.py Posts.csv > evolucion_tecnologias.out
```

**Salida esperada**:

Tag, tendencia, promedio_delta (con 1 decimal, el signo sólo en los negativos), meses_considerados. Todo separado por tabuladores.

```
apache  CRECIMIENTO     92.3    6
apache-cordova  DECLIVE -8.3    6
apache-netbeans CRECIMIENTO     19.4    6
apache-pdfbox   ESTABLE 0.0     3
...
```

## Evaluación y entrega

### Criterios de evaluación

Se tendrá en cuenta, por orden de importancia: Que el código funcione correctamente y produzca los resultados esperados; que se usen buenas prácticas de programación y eficiencia en MapReduce; y que el código tenga suficientes comentarios y esté bien estructurado. Se valorará positivamente la construcción de tests para probar que los ejercicios funcionan correctamente.

### Entrega

Entregar en la tarea del Aula Virtual un archivo comprimido `.zip` o `.tar.gz` con el código fuente de los ejercicios realizados y un pequeño informe explicando qué se ha realizado en cada paso de Map-Reduce para cada ejercicio. Opcionalmente, se pueden incluir programas de prueba (*unit tests*) para verificar el correcto funcionamiento de los ejercicios (se muestra un ejemplo en las plantillas de código más abajo).

## ANEXO: Plantillas de código

A continuación se muestran plantillas útiles para empezar los ejercicios. Nótese cómo, para evitar problemas con los datos, se utiliza la propia librería `csv` de Python para parsear las líneas de entrada (aunque sea sólo una línea, que es lo que recibe cada llamada al *mapper*).

Las clases `TextValueProtocol` y `TextProtocol` permiten que los datos de entrada y salida se traten como texto plano, evitando problemas con los tipos por defecto de `mrjob`. En general, como la salida será normalmente una clave de texto y un valor de texto separados por tabulador, este formato acepta cadenas de caracteres y produce el resultado correcto.

En los pasos intermedios, MrJob, al utilizar Hadoop (que trata todo en binario), se pueden utilizar como claves y valores tipos estándar de python como tuplas, listas, arrays, etc. (se puede ver en los ejemplos de más abajo)


### Plantilla básica MRJob

```python
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol, TextValueProtocol
import csv

class MiEjercicio(MRJob):
    INPUT_PROTOCOL = TextValueProtocol
    OUTPUT_PROTOCOL = TextProtocol
    
    def mapper(self, key, value):
        """Procesar cada línea del CSV"""
        try:
            row = next(csv.reader([value]))
            
            # Skip header
            if row and row[0].strip().lower() == 'id':
                return
                
            # Extraer campos necesarios
            campo1 = row[POSICION].strip()
            
            # Lógica del ejercicio
            if condicion:
                yield clave, valor
                
        except (ValueError, IndexError, csv.Error):
            return  # Ignorar líneas malformadas
    
    def combiner(self, key, values):
        """Combinar localmente (opcional pero recomendado)"""
        yield key, ...
        
    def reducer(self, key, values):
        """Agregación final"""
        # Realizar cálculos
        # ...
        yield key, ...

if __name__ == '__main__':
    MiEjercicio.run()
```


### Plantilla para ejercicios multi-paso

```python
from mrjob.step import MRStep

# Paso 1
class MiEjercicio(MRJob):

    def steps(self) -> list[MRStep]:
        return [
            MRStep(mapper=self.mapper1,
                    reducer=self.reducer1),
            MRStep(mapper=self.mapper2,
                    reducer=self.reducer2),
            ...  # Más pasos si es necesario
        ]

    def mapper1(self, key, value):
        # Procesar datos originales
        yield clave_intermedia, valor_intermedio
    
    def reducer1(self, key, values):
        yield key, proceso(values)

    # Paso 2
    def mapper2(self, key, value):
        # Procesar salida del paso 1
        yield nueva_clave, nuevo_valor

    def reducer2(self, key, values):
        yield key, agregacion_final(values)
```



### Ejemplo de construcción de tests

Para realizar los tests, supondremos que tenemos la salida de cada ejercicio en su fichero `.out`. Primero, hay que asegurarse de que se tiene instalado `pandas` y `pytest` en el entorno virtual:

```bash
pip install pandas pytest
```

El archivo de ejemplo `test_ejercicio1.py` muestra cómo comparar la salida del ejercicio 1 con una implementación de referencia usando `pandas`:

```python
# test_count_answers.py
from pandas import DataFrame, Series, read_csv, notna


def test_so_count_answers() -> None:
    """Compara so_count_answers.py con implementación pandas de referencia"""
    output_file: str = "so_count_answers.out"
    posts_file: str = "Posts.csv"

    # Usar el resultado del fichero generado previamente
    mrjob_results: DataFrame = read_csv(
        output_file, sep="\t", header=None, names=["QuestionId", "AnswerCount"]
    )

    # Implementación de referencia con pandas
    df: DataFrame = read_csv(posts_file)

    # Filtrar respuestas (PostTypeId = 2) con ParentId válido
    answers: DataFrame = df[df["PostTypeId"] == 2]

    # Contar respuestas por pregunta (ParentId ya es el índice)
    answer_counts: Series = answers["ParentId"].value_counts(sort=False)

    # Comparar resultados
    assert len(mrjob_results) == len(
        answer_counts
    ), f"Diferente número de preguntas: MRJob={len(mrjob_results)}, Pandas={len(answer_counts)}"

    for question_id, mrjob_count in mrjob_results.itertuples(index=False):
        pandas_count: int = answer_counts.get(question_id, 0)
        assert (
            mrjob_count == pandas_count
        ), f"Diferencia en pregunta {question_id}: MRJob={mrjob_count}, Pandas={pandas_count}"

    # Verificar que el campo AnswerCount de cada pregunta coincide con el conteo real
    questions: DataFrame = df[df["PostTypeId"] == 1]
    questions_with_answers: DataFrame = questions[questions["Id"].isin(answer_counts.index)]

    for _, question in questions_with_answers.iterrows():
        question_id: int = int(question["Id"])
        declared_count: int = int(question["AnswerCount"]) if notna(question["AnswerCount"]) else 0
        actual_count: int = answer_counts.get(question_id, 0)

        assert declared_count == actual_count, (
            f"El campo AnswerCount no coincide para pregunta {question_id}: "
            f"AnswerCount={declared_count}, conteo real={actual_count}"
        )

    print(f"Comparación exitosa: {len(mrjob_results)} preguntas con respuestas")
    print(f"Verificación AnswerCount: {len(questions_with_answers)} preguntas verificadas")
```

Finalmente, para probarlo:

```bash
pytest test_count_answers.py
```